In [319]:
from keplergl import KeplerGl
import pandas as pd
import geojson
import random

line_number = 1

In [320]:
line_1_route_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/routes.csv')
line_1_route_df = line_1_route_df[line_1_route_df['route_short_name'] == str(line_number)]

line_1_route_df

,route_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,2,1,GARE DE L'OUEST - STOCKEL,1,C4008F,FFFFFF


In [321]:
line_1_trips_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/trips.csv')
line_1_trips_df = line_1_trips_df.merge(line_1_route_df)
line_1_trips_list = line_1_trips_df['trip_id'].unique().tolist()

line_1_trips_list[:8]

[112585809234578052,
 112585811234578052,
 112585813234578052,
 112585814234578052,
 112585815234578052,
 112585816234578052,
 112585817234578052,
 112585818234578052]

In [322]:
line_1_trip_stops = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/stop_times.csv')
line_1_trip_stops = line_1_trip_stops[line_1_trip_stops['trip_id'].isin(line_1_trips_list)]
line_1_trip_stops = line_1_trip_stops.groupby('trip_id')['stop_id'].apply(lambda x: tuple(x.tolist())).reset_index(name='stop_sequence')

line_1_trip_stops

,trip_id,stop_sequence
0,112585809234578052,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809..."
1,112585811234578052,"(8162, 8151, 8141, 8131, 8121, 8111, 8101, 809..."
2,112585813234578052,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803..."
3,112585814234578052,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803..."
4,112585815234578052,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803..."
...,...,...
2133,113423393237269601,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809..."
2134,113423395237269601,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803..."
2135,113423397237269601,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809..."
2136,113423399237269601,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803..."


In [323]:
line_1_trips_per_sequence = line_1_trip_stops.groupby(['stop_sequence'])['trip_id'].count().reset_index(name='count')
line_1_trips_per_sequence = line_1_trips_per_sequence.sort_values(by=['count'], ascending=False)

line_1_trips_per_sequence

,stop_sequence,count
8,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",830
3,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809...",817
5,"(8162, 8151, 8141, 8131, 8121, 8111, 8101, 809...",226
9,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",219
0,"(8072, 8082, 8092, 8102, 8112, 8122, 8132, 814...",18
4,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809...",10
2,"(8151, 8141, 8131, 8121, 8111, 8101, 8091, 808...",8
1,"(8072, 8082, 8092, 8102, 8112, 8122, 8132, 814...",7
7,"(8642, 8652, 8662, 8672, 8682, 8692, 8702, 871...",2
6,"(8642, 8652, 8662, 8672, 8682, 8692, 8702, 871...",1


In [324]:
terminals = line_1_route_df.iloc[0, 2]
terminals = terminals.split(' - ')

terminals

["GARE DE L'OUEST", 'STOCKEL']

In [325]:
terminal_id_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/stops.csv')
terminal_id_df = terminal_id_df[terminal_id_df['stop_name'].isin(terminals)]
terminal_id_df = terminal_id_df.groupby(['stop_name'])['stop_id'].apply(lambda x: x.tolist()).reset_index(name='stop_ids')

terminal_id_df

,stop_name,stop_ids
0,GARE DE L'OUEST,"[1145, 3230, 3230F, 3253, 3253F, 8381, 8382, 8..."
1,STOCKEL,"[1252, 1302, 6474F, 6475F, 8161, 8162, 29]"


In [326]:
terminal_id_lists = []
for _, row in terminal_id_df.iterrows():
    terminal_id_lists.append(row['stop_ids'])

terminal_id_lists

[['1145',
  '3230',
  '3230F',
  '3253',
  '3253F',
  '8381',
  '8382',
  '8731',
  '8732',
  '8733',
  '2'],
 ['1252', '1302', '6474F', '6475F', '8161', '8162', '29']]

In [327]:
valid_trips_df = pd.DataFrame({'stop_sequence': [], 'count': []})
for _, row in line_1_trips_per_sequence.iterrows():
    if any(x in row['stop_sequence'][:2] for x in terminal_id_lists[0]) and \
        any(x in row['stop_sequence'][-2:] for x in terminal_id_lists[1]):
        valid_trips_df.loc[len(valid_trips_df.index)] = row
    elif any(x in row['stop_sequence'][:2] for x in terminal_id_lists[1]) and \
        any(x in row['stop_sequence'][-2:] for x in terminal_id_lists[0]):
        valid_trips_df.loc[len(valid_trips_df.index)] = row

valid_trips_df

,stop_sequence,count
0,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",830
1,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809...",817
2,"(8162, 8151, 8141, 8131, 8121, 8111, 8101, 809...",226
3,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",219


In [328]:
valid_trip_ids = valid_trips_df.merge(line_1_trip_stops)
valid_trip_ids = valid_trip_ids.groupby('stop_sequence', as_index=False).first()

valid_trip_ids = valid_trip_ids.sort_values(by=['count'], ascending=False)

valid_trip_ids

,stop_sequence,count,trip_id
2,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",830,112585816234578052
0,"(8161, 8151, 8141, 8131, 8121, 8111, 8101, 809...",817,112585809234578052
1,"(8162, 8151, 8141, 8131, 8121, 8111, 8101, 809...",226,112585811234578052
3,"(8733, 8742, 8292, 8282, 8272, 8012, 8022, 803...",219,112585813234578052


In [329]:
line_1_stops_list = valid_trip_ids.iloc[0,0]

line_1_stops_list

('8733',
 '8742',
 '8292',
 '8282',
 '8272',
 '8012',
 '8022',
 '8032',
 '8042',
 '8052',
 '8062',
 '8072',
 '8082',
 '8092',
 '8102',
 '8112',
 '8122',
 '8132',
 '8142',
 '8152',
 '8161')

In [330]:
line_stops_df = pd.DataFrame({'stop_id': line_1_stops_list})
line_stops_df = line_stops_df.merge(stops_df)
line_stops_df = line_stops_df[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
line_stops_df['scores'] = [random.randint(1,11) for i in range(line_stops_df.shape[0])]

line_stops_df

,stop_id,stop_name,stop_lat,stop_lon,scores
0,8733,GARE DE L'OUEST,50.848999,4.320948,10
1,8742,BEEKKANT,50.853386,4.322974,11
2,8292,ETANGS NOIRS,50.857125,4.333143,8
3,8282,COMTE DE FLANDRE,50.854705,4.340542,5
4,8272,SAINTE-CATHERINE,50.851900,4.348012,10
5,8012,DE BROUCKERE,50.849821,4.352486,8
6,8022,GARE CENTRALE,50.846349,4.358776,4
7,8032,PARC,50.845688,4.362979,6
8,8042,ARTS-LOI,50.845333,4.369620,5
9,8052,MAELBEEK,50.843654,4.377588,11


In [331]:
shapes_1_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/shapes.csv')

valid_shapes_1_df = pd.DataFrame({'trip_id': [valid_trip_ids.iloc[0,2]]})
valid_shapes_1_df = valid_shapes_1_df.merge(trips_df)
valid_shapes_1_df = valid_shapes_1_df.merge(shapes_1_df)

valid_shapes_1_df = valid_shapes_1_df.groupby(['shape_id'])[['shape_pt_lon', 'shape_pt_lat']].apply(lambda x: geojson.LineString(x.values.tolist())).reset_index(name='geojson')

valid_shapes_1_df

,shape_id,geojson
0,001m0080,"{'type': 'LineString', 'coordinates': [[4.3209..."


In [345]:
valid_shapes_df_list = [valid_shapes_1_df]

def calculate_stops_and_shapes_df(line_number):
    line_route_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/routes.csv')
    line_route_df = line_route_df[line_route_df['route_short_name'] == str(line_number)]

    line_trips_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/trips.csv')
    line_trips_df = line_trips_df.merge(line_route_df)
    line_trips_list = line_trips_df['trip_id'].unique().tolist()

    line_trip_stops = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/stop_times.csv')
    line_trip_stops = line_trip_stops[line_trip_stops['trip_id'].isin(line_trips_list)]
    line_trip_stops = line_trip_stops.groupby('trip_id')['stop_id'].apply(lambda x: tuple(x.tolist())).reset_index(name='stop_sequence')

    line_trips_per_sequence = line_trip_stops.groupby(['stop_sequence'])['trip_id'].count().reset_index(name='count')
    line_trips_per_sequence = line_trips_per_sequence.sort_values(by=['count'], ascending=False)

    terminals = line_route_df.iloc[0, 2]
    terminals = terminals.split(' - ')
    for i in range(len(terminals)):
        match terminals[i]:
            case 'BRUSSELS CITY':
                terminals[i] = 'TRONE'
            case 'GROOT-BIJGAARDEN':
                terminals[i] = 'HUNDERENVELD'
            case 'CIMETIERE DE JETTE':
                terminals[i] = 'BIENFAITEURS'
            case 'HOPITAL MILITAIRE':
                terminals[i] = 'HOP. MILITAIRE'
            case 'HOPITAL ETTERBEEK-IXELLES':
                terminals[i] = 'HOP. ETT.-IXELLES'
            case 'CIMETIERE DE BRUXELLES':
                terminals[i] = 'CIM. DE BRUXELLES'
            case 'UCCLE STALLE':
                terminals[i] = 'UCCLE-STALLE'
            case 'DE BROUCKERE':
                if line_number == 66:
                    terminals[i] = 'GARE CENTRALE'
            case _:
                pass

    terminal_id_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/stops.csv')
    terminal_id_df = terminal_id_df[terminal_id_df['stop_name'].isin(terminals)]
    terminal_id_df = terminal_id_df.groupby(['stop_name'])['stop_id'].apply(lambda x: x.tolist()).reset_index(name='stop_ids')

    terminal_id_lists = []
    for _, row in terminal_id_df.iterrows():
        terminal_id_lists.append(row['stop_ids'])

    valid_trips_df = pd.DataFrame({'stop_sequence': [], 'count': []})
    for _, row in line_trips_per_sequence.iterrows():
        if any(x in row['stop_sequence'][:2] for x in terminal_id_lists[0]) and \
            any(x in row['stop_sequence'][-2:] for x in terminal_id_lists[1]):
            valid_trips_df.loc[len(valid_trips_df.index)] = row
        elif any(x in row['stop_sequence'][:2] for x in terminal_id_lists[1]) and \
            any(x in row['stop_sequence'][-2:] for x in terminal_id_lists[0]):
            valid_trips_df.loc[len(valid_trips_df.index)] = row

    valid_trip_ids = valid_trips_df.merge(line_trip_stops)
    valid_trip_ids = valid_trip_ids.groupby('stop_sequence', as_index=False).first()

    valid_trip_ids = valid_trip_ids.sort_values(by=['count'], ascending=False)

    line_stops_list = valid_trip_ids.iloc[0,0]

    line_stops_df = pd.DataFrame({'stop_id': line_stops_list})
    line_stops_df = line_stops_df.merge(stops_df)
    line_stops_df = line_stops_df[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
    line_stops_df['scores'] = [random.randint(1,11) for i in range(line_stops_df.shape[0])]

    shapes_df = pd.read_csv('../../data/Project Data-20221123/gtfs3Sept/shapes.csv')

    valid_shapes_df = pd.DataFrame({'trip_id': [valid_trip_ids.iloc[0,2]]})
    valid_shapes_df = valid_shapes_df.merge(trips_df)
    valid_shapes_df = valid_shapes_df.merge(shapes_df)

    valid_shapes_df = valid_shapes_df.groupby(['shape_id'])[['shape_pt_lon', 'shape_pt_lat']].apply(lambda x: geojson.LineString(x.values.tolist())).reset_index(name='geojson')

    return line_stops_df, valid_shapes_df

metros = [1, 2, 5, 6]
trams = [4, 8, 9, 19, 39, 44, 51, 55, 62, 81, 82, 92, 93, 97]
interruped_trams = [3, 7, 25]
buses = [12, 13, 14, 17, 20, 21, 27, 28, 29, 33, 34, 36, 37, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 52, 53, 54, 56, 58, 59, 60, 61, 63, 64, 65, 66, 71, 72, 74, 75, 79, 80, 83, 86, 87, 88, 89, 95]
unavailable_buses = [73, 90]
vehicles = metros + trams + buses

trial_numbers = sorted(vehicles)

for i in trial_numbers:
    print(i)
    line_stops_df = pd.concat([line_stops_df, calculate_stops_and_shapes_df(i)[0]], axis=0)
    valid_shapes_df_list.append(calculate_stops_and_shapes_df(i)[1])

1
2
4
5
6
8
9
12
13
14
17
19
20
21
27
28
29
33
34
36
37
38
39
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
58
59
60
61
62
63
64
65
66
71
72
74
75
79
80
81
82
83
86
87
88
89
92
93
95
97


In [351]:
map_1 = KeplerGl()

map_1.add_data(data=line_stops_df, name='STIB Stops')

print(len(valid_shapes_df_list))

for i, item in enumerate(valid_shapes_df_list[:-1]):
    map_1.add_data(data=item, name=f'itinerary {trial_numbers[i]}')

map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
66


KeplerGl(data={'STIB Stops': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …